In [21]:
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D

from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np

In [137]:
def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(4, activation='softmax')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [138]:
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [152]:
# define model
model = define_model()
# create data generator
# create data generators
train_datagen = ImageDataGenerator(featurewise_center=True,rescale=1.0/255.0,width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
test_datagen = ImageDataGenerator(featurewise_center=True,rescale=1.0/255.0)
# specify imagenet mean values for centering
datagen.mean = [123.68, 116.779, 103.939]
# prepare iterator
train_it = train_datagen.flow_from_directory('/kaggle/input/data-for-rps/image_data/train/',
    class_mode='categorical', batch_size=64, target_size=(224, 224))
test_it = test_datagen.flow_from_directory('/kaggle/input/data-for-rps/image_data/test/',
    class_mode='categorical', batch_size=64, target_size=(224, 224))
# fit model
history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
    validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=1)
# evaluate model
_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
print('> %.3f' % (acc * 100.0))
model.save('accuracy - %.3f' % (acc * 100.0) + '.h5')
# learning curves
summarize_diagnostics(history)


Found 800 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Epoch 1/10


/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


13/13 [==============================] - 18s 1s/step - loss: 1.3213 - accuracy: 0.3913 - val_loss: 1.1179 - val_accuracy: 0.6050
Epoch 2/10
13/13 [==============================] - 16s 1s/step - loss: 0.8626 - accuracy: 0.6900 - val_loss: 1.2572 - val_accuracy: 0.5350
Epoch 3/10
13/13 [==============================] - 17s 1s/step - loss: 0.5772 - accuracy: 0.8225 - val_loss: 0.9090 - val_accuracy: 0.7475
Epoch 4/10
13/13 [==============================] - 16s 1s/step - loss: 0.3767 - accuracy: 0.8963 - val_loss: 0.8232 - val_accuracy: 0.6850
Epoch 5/10
13/13 [==============================] - 17s 1s/step - loss: 0.3123 - accuracy: 0.9275 - val_loss: 0.9908 - val_accuracy: 0.5525
Epoch 6/10
13/13 [==============================] - 16s 1s/step - loss: 0.2430 - accuracy: 0.9388 - val_loss: 0.6101 - val_accuracy: 0.6625
Epoch 7/10
13/13 [==============================] - 16s 1s/step - loss: 0.2038 - accuracy: 0.9575 - val_loss: 1.2106 - val_accuracy: 0.6500
Epoch 8/10
13/13 [=============

In [50]:
train_it.class_indices

{'none': 0, 'paper': 1, 'rock': 2, 'scissors': 3}

In [149]:
import cv2
img = cv2.imread('/kaggle/input/data-for-rps/image_data/test/rock/63.jpg')
img = cv2.resize(img, (224, 224))
pred = model.predict(np.array([img]))

In [150]:
pred
move_code = np.argmax(pred[0])
move_code

2